In [13]:
import csv
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfTransformer

In [52]:
train = pd.read_csv('reviews_tr.csv')
train = shuffle(train)

In [53]:
text_tr1 = train['text']
labels1 = train['rating']

In [54]:
vectorizer = CountVectorizer()
data1 = vectorizer.fit_transform(text_tr1)

In [55]:
labels1 = np.array(labels1)
labels1 = labels1.reshape(labels1.shape[0],1)
for i in range(labels1.shape[0]):
    if labels1[i] == 0:
        labels1[i] = -1

In [56]:
w = np.zeros((data1.shape[1]+1,1))

In [57]:
for i in range(data1.shape[0]):
    tmp = (data1[i]*w[1:] + w[0]) * labels1[i][0]
    if tmp <= 0:
        label = labels1[i][0]
        w[1:] += label*data1[i].T
        w[0] += labels1[i]

In [58]:
train = shuffle(train)
text_tr2 = train['text']
labels2 = train['rating']
vectorizer2 = CountVectorizer()
data2 = vectorizer2.fit_transform(text_tr2)
labels2 = np.array(labels2)
labels2 = labels2.reshape(labels2.shape[0],1)
for i in range(labels2.shape[0]):
    if labels2[i] == 0:
        labels2[i] = -1

In [59]:
w_final = w / (data2.shape[0]+1)
for i in range(data2.shape[0]):
    tmp = (data2[i]*w[1:] + w[0]) * labels2[i][0]
    if tmp <= 0:
        label = labels2[i][0]
        w[1:] += label*data2[i].T
        w[0] += labels2[i]
    w_final += w / (data2.shape[0]+1)

In [60]:
w_final

array([[ 7.24028576],
       [-8.31621568],
       [-8.22193678],
       ...,
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [73]:
test = pd.read_csv('reviews_te.csv')
test = shuffle(test)
text_te = test['text']
labels_te = test['rating']
data_te = vectorizer_te.transform(text_te)
labels_te = np.array(labels_te)
labels_te = labels_te.reshape(labels_te.shape[0],1)
for i in range(labels_te.shape[0]):
    if labels_te[i] == 0:
        labels_te[i] = -1

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

In [67]:
w_final.shape

(207393, 1)

In [70]:
count = 0
for i in range(data2.shape[0]):
    tmp = (data2[i]*w[1:] + w[0]) * labels2[i][0]
    if tmp <= 0:
        count += 1
error = float(count / data2.shape[0])

In [71]:
error

0.0

In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import random
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def first_round(data,labels):
    w = np.zeros((data.shape[1]+1,1))
    for i in range(data.shape[0]):
        tmp = (data[i] * w[1:] + w[0]) * labels[i][0]
        if tmp <= 0:
            w[1:] += labels[i][0] * data[i].T
            w[0] += labels[i]
    return w


def second_round(data,labels,w):
    countlist = list(range(data.shape[0]))
    random.shuffle(countlist)
    w_final = w / (data.shape[0]+1)
    for i in countlist:
        tmp = (data[i] * w[1:] + w[0]) * labels[i][0]
        if tmp <= 0:
            w[1:] += labels[i][0] * data[i].T
            w[0] += labels[i]
        w_final += w / (data.shape[0]+1)
    return w_final


def count_error(data,labels,w_final):
    count = 0
    for i in range(data.shape[0]):
        tmp = (data[i] * w_final[1:] + w_final[0]) * labels[i][0]
        if tmp <= 0:
            count += 1
    error = float(count / data.shape[0])
    return error

In [4]:
train = pd.read_csv('reviews_tr.csv')
text = train['text']
labels = train['rating']
vectorizer = CountVectorizer()
data = vectorizer.fit_transform(text)
labels = np.array(labels)
labels = labels.reshape(labels.shape[0],1)
for i in range(labels.shape[0]):
    if labels[i] == 0:
        labels[i] = -1

In [ ]:
test = pd.read_csv('reviews_te.csv')
test_text = test['text']
testlabels = test['rating']
testdata = vectorizer.transform(test_text)
testlabels = np.array(testlabels)
testlabels = testlabels.reshape(testlabels.shape[0],1)
for i in range(testlabels.shape[0]):
    if testlabels[i] == 0:
        testlabels[i] = -1

In [72]:
w = first_round(data,labels)
w_final = second_round(data,labels,w)
error = count_error(data,labels,w_final)
print(error)

In [12]:
error_test = count_error(testdata,testlabels,w_final)
print(error_test)

0.10546291726279355


In [15]:
w_list = []
for i in range(w_final.shape[0]):
    w_list.append(w_final[i][0])
print(w_list)

NameError: name 'w_final' is not defined

In [26]:
np.argpartition(w_list, -10)[-10:]

array([ 90115, 202496, 137900,  78575,  94263,  56663, 167373,  66810,
       138918,  87294])

In [14]:
word = vectorizer.get_feature_names()
idxh = np.argpartition(w_list, -10)[-10:]
idxl = np.argpartition(w_list, 10)[:10]
high = []
for i in idxh:
    high.append(word[i])
low = []
for i in idxl:
    low.append(word[i])
high.sort()
low.sort()
print(high)
print(low)

NameError: name 'w_list' is not defined

In [74]:
count = 0
idx = []
countlist = list(range(data.shape[0]))
random.shuffle(countlist)

for i in countlist: 
    if count < 2:
        tmp = (data[i] * w[1:] + w[0]) * labels[i][0]
        if tmp <= 0:
            count += 1
            print(text[i])
            datatmp = data[i].toarray()
            #print(type(datatmp))
            w_list2 = []
            for i in range(w.shape[0]):
                w_list2.append(w[i][0])
            wx = np.multiply(datatmp, w_list2[1:])
            #print(type(wx))
            #print(wx)
            wx = wx.tolist()
            wx = wx[0]
            print(type(wx))
            idxh = np.argpartition(wx, -10)[-10:]
            idxl = np.argpartition(wx, 10)[:10]
            print(type(idxh))
            #print(idxh)
            high2 = []
            highvalue = []
            for i in idxh:
                high2.append(word[i])
                highvalue.append(wx[i])
            low2 = []
            lowvalue = []
            for i in idxl:
                low2.append(word[i])
                lowvalue.append(wx[i])
            print(high2)
            print(highvalue)
            print(low2)
            print(lowvalue)
            w[1:] += labels[i][0] * data[i].T
            w[0] += labels[i]
    else:
        break

everytime i go to las vegas i would find a morning to have dim sum at ping pang pong even i live in la a place have tons of dim sum restaurant for my taste preference ping pang pong offers much better taste than any other dim sum place in la you should go early unless you want to wait for at least an hour
<class 'list'>
<class 'numpy.ndarray'>
['in', 'everytime', 'early', 'place', 'you', 'to', 'preference', 'dim', 'vegas', 'wait']
[16.0, 17.0, 17.0, 24.0, 36.0, 39.0, 23.0, 27.0, 19.0, 32.0, -54.0, -51.0, -25.0, -28.0, -27.0, -40.0, -30.0, -49.0, -30.0, -39.0]
['taste', 'sum', 'should', 'want', 'offers', 'least', 'other', 'hour', 'la', 'unless']
[]
i love dim sum those little small bite sized dishes always get me i enjoyed eating at the cathay house restaurant with my boyfriend and his mother it was a good place to hit up my favorite dish here was the steamed bums and those sesame balls that are like extremely dense but delicious i left feeling like i had a food baby inside of me becaus

In [8]:
from toolz import itertoolz, compose
from toolz.curried import map as cmap, sliding_window, pluck

class SkipGramVectorizer(CountVectorizer):
    def build_analyzer(self):    
        preprocess = self.build_preprocessor()
        stop_words = self.get_stop_words()
        tokenize = self.build_tokenizer()
        return lambda doc: self._word_skip_grams(
                compose(tokenize, preprocess, self.decode)(doc),
                stop_words)

    def _word_skip_grams(self, tokens, stop_words=None):
        # handle stop words
        if stop_words is not None:
            tokens = [w for w in tokens if w not in stop_words]

        return compose(cmap(' '.join), pluck([0, 2]), sliding_window(3))(tokens)

In [9]:
vect = SkipGramVectorizer()
data_skip = vect.fit_transform(text)
print(data_skip.shape)

(1000000, 9755470)


In [10]:
w_skip = first_round(data_skip,labels)

In [13]:
w_final_skip = second_round(data_skip,labels,w_skip)
print(w_final_skip)

KeyboardInterrupt: 

In [ ]:
error_skip_tr = count_error(data_skip,labels,w_final_skip)
print(error_skip_tr)

In [77]:
type(data_skip)

__main__.SkipGramVectorizer

In [78]:
vectorizer_idf = TfidfVectorizer()
data_idf = vectorizer_idf.fit_transform(text)

In [79]:
w_idf = first_round(data_idf,labels)
w_final_idf = second_round(data_idf,labels,w_idf)
error_idf_tr = count_error(data_idf,labels,w_final_idf)
print(error_idf_tr)

0.095156


In [80]:
testdata_idf = vectorizer_idf.transform(test_text)
error_idf_te = count_error(testdata_idf,testlabels,w_final_idf)
print(error_idf_te)

0.10586276482091203


In [81]:
bigram = CountVectorizer(ngram_range=(2, 2))
data_bi = bigram.fit_transform(text)
w_bi = first_round(data_bi,labels)
w_final_bi = second_round(data_bi,labels,w_bi)
error_bi_tr = count_error(data_bi,labels,w_final_bi)
print(error_bi_tr)

0.049628


In [85]:
testdata_bi = bigram.transform(test_text)
error_bi_te = count_error(testdata_bi,testlabels,w_final_bi)
print(error_bi_te)

0.10005560380105084


In [5]:
trigram = CountVectorizer(ngram_range=(3, 3))

In [6]:
data_tri = trigram.fit_transform(text)

In [7]:
w_tri = first_round(data_tri,labels)

KeyboardInterrupt: 

In [ ]:
w_final_tri = second_round(data_tri,labels,w_tri)

In [ ]:
error_tri_tr = count_error(data_tri,labels,w_final_tri)
print(error_tri_tr)

In [ ]:
testdata_tri = trigram.transform(test_text)
error_tri_te = count_error(testdata_tri,testlabels,w_final_tri)
print(error_tri_te)